# Goal is to display all workouts
#### * Query database
#### * Transform models into viewmodels
#### * Display data to user

In [45]:
import sqlite3
import json
from datetime import datetime

In [2]:
DATABASE = 'workout.db'

### Define function to query database

In [3]:
def query_db(query, args=(), one=False):
    conn = sqlite3.connect(DATABASE)
    conn.row_factory = sqlite3.Row
    cur = conn.execute(query, args)
    rv = cur.fetchall()
    conn.commit()
    cur.close()
    return (rv[0] if rv else None) if one else rv

### Query all workouts & topsets completed by Gabe(PersonId=1)

In [70]:
topsets = query_db("""
    SELECT 
        P.PersonId, 
        P.Name AS PersonName, 
        W.WorkoutId, 
        W.StartDate, 
        T.TopSetId, 
        E.ExerciseId, 
        E.Name AS ExerciseName, 
        T.Repetitions, 
        T.Weight
    FROM Person P
         LEFT JOIN Workout W ON P.PersonId=W.PersonId
         LEFT JOIN TopSet T ON W.WorkoutId=T.WorkoutId
         LEFT JOIN Exercise E ON T.ExerciseId=E.ExerciseId
    WHERE P.PersonId=?""", [1])

for topset in topsets:
    print(f'WorkoutId: {topset["WorkoutId"]}, PersonId: {topset["PersonId"]} Person:{topset["PersonName"]}, StartDate: {topset["StartDate"]}, TopSetId: {topset["TopSetId"]}, ExerciseId: {topset["ExerciseId"]}, Exercise: {topset["ExerciseName"]}, Repetitions: {topset["Repetitions"]}, Weight: {topset["Weight"]}kg')

WorkoutId: 35, PersonId: 1 Person:Gabe, StartDate: 2022-01-13, TopSetId: 61, ExerciseId: 2, Exercise: Bench, Repetitions: 8, Weight: 50kg
WorkoutId: 36, PersonId: 1 Person:Gabe, StartDate: 2022-01-18, TopSetId: 62, ExerciseId: 1, Exercise: Squat, Repetitions: 7, Weight: 55kg
WorkoutId: 36, PersonId: 1 Person:Gabe, StartDate: 2022-01-18, TopSetId: 63, ExerciseId: 4, Exercise: Hotep, Repetitions: 4, Weight: 25kg
WorkoutId: 37, PersonId: 1 Person:Gabe, StartDate: 2022-02-08, TopSetId: 64, ExerciseId: 2, Exercise: Bench, Repetitions: 5, Weight: 55kg
WorkoutId: 37, PersonId: 1 Person:Gabe, StartDate: 2022-02-08, TopSetId: 65, ExerciseId: 3, Exercise: Deadlift, Repetitions: 6, Weight: 95kg
WorkoutId: 38, PersonId: 1 Person:Gabe, StartDate: 2022-02-15, TopSetId: 66, ExerciseId: 1, Exercise: Squat, Repetitions: 3, Weight: 65kg
WorkoutId: 38, PersonId: 1 Person:Gabe, StartDate: 2022-02-15, TopSetId: 67, ExerciseId: 4, Exercise: Hotep, Repetitions: 6, Weight: 25kg
WorkoutId: 39, PersonId: 1 Pers

### Transform entity to view model with the following schema
<pre>
{   "PersonId": 1,
    "PersonName": "Gabe",  
    "Exercises": [{"ExerciseIs": 1, "ExerciseName": "Squat"}, {"ExerciseIs": 2, "ExerciseName": "Bench"}]  
    "Workouts": [  
        {  
            "WorkoutId": 1,
            "StartDate": "2022-06-29 00:00:00.000",  
            "TopSets": [  
                {  
                    "ExerciseId": 1,
                    "ExerciseName": "Squat",  
                    "Weight": 80,  
                    "Repetitions": 8  
                },  
                {  
                    "ExerciseId": 2,
                    "ExerciseName": "Bench",  
                    "Weight": 60,  
                    "Repetitions": 4  
                }  
            ]  
        },  
        {
            "WorkoutId": 2,
            "StartDate": "2022-07-01 00:00:00.000",  
            "TopSets": [  
                {  
                    "ExerciseId": 1,
                    "ExerciseName": "Squat",  
                    "Weight": 85,  
                    "Repetitions": 4  
                },  
                {  
                    "ExerciseId": 2,
                    "ExerciseName": "Bench",  
                    "Weight": 65,  
                    "Repetitions": 5  
                }  
            ]  
        }  
    ]  
} 
</pre>

#### 1. Extract name from rows

In [37]:
def get_name(topsets):
    return topsets[0]['PersonName']

get_name(topsets)

'Gabe'

#### 2. Group rows into workouts using WorkoutId column

In [15]:
def get_workouts(topsets):
    # Get all unique workout_ids (No duplicates)
    workout_ids = set([t['WorkoutId'] for t in topsets])

    # Group topsets into workouts
    workouts = []
    for workout_id in workout_ids:
        topsets_in_workout = [t for t in topsets if t['WorkoutId'] == workout_id]
        workouts.append({
            'WorkoutId': workout_id,
            'StartDate': topsets_in_workout[0]['StartDate'],
            'TopSets': [{"TopSetId": t['TopSetId'], "ExerciseId": t['ExerciseId'], "ExerciseName": t['ExerciseName'], "Weight": t['Weight'], "Repetitions": t['Repetitions'] } for t in topsets_in_workout]
        })
    return workouts

In [16]:
print(json.dumps(get_workouts(topsets), indent=2))

[
  {
    "WorkoutId": 35,
    "StartDate": "2022-01-13",
    "TopSets": [
      {
        "TopSetId": 61,
        "ExerciseId": 2,
        "ExerciseName": "Bench",
        "Weight": 50,
        "Repetitions": 8
      }
    ]
  },
  {
    "WorkoutId": 36,
    "StartDate": "2022-01-18",
    "TopSets": [
      {
        "TopSetId": 62,
        "ExerciseId": 1,
        "ExerciseName": "Squat",
        "Weight": 55,
        "Repetitions": 7
      },
      {
        "TopSetId": 63,
        "ExerciseId": 4,
        "ExerciseName": "Hotep",
        "Weight": 25,
        "Repetitions": 4
      }
    ]
  },
  {
    "WorkoutId": 37,
    "StartDate": "2022-02-08",
    "TopSets": [
      {
        "TopSetId": 64,
        "ExerciseId": 2,
        "ExerciseName": "Bench",
        "Weight": 55,
        "Repetitions": 5
      },
      {
        "TopSetId": 65,
        "ExerciseId": 3,
        "ExerciseName": "Deadlift",
        "Weight": 95,
        "Repetitions": 6
      }
    ]
  },
  {
    "Workout

In [17]:
def get_all_exercises_from_topsets(topsets):
    exercise_ids = set([t['ExerciseId'] for t in topsets])
    exercises = []
    for exercise_id in exercise_ids:
        exercises.append({
            'ExerciseId': exercise_id,
            'ExerciseName': next((t['ExerciseName'] for t in topsets if t['ExerciseId'] == exercise_id), 'Unknown')
        })
    return exercises

get_all_exercises_from_topsets(topsets)


[{'ExerciseId': 1, 'ExerciseName': 'Squat'},
 {'ExerciseId': 2, 'ExerciseName': 'Bench'},
 {'ExerciseId': 3, 'ExerciseName': 'Deadlift'},
 {'ExerciseId': 4, 'ExerciseName': 'Hotep'}]

### 3. Compose functions together to acheive final model

In [18]:
def convert_to_view_model(topsets):
    return {
        'PersonId': next((t['PersonId'] for t in topsets), -1),
        'PersonName': next((t['PersonName'] for t in topsets), -1),
        'Exercises': get_all_exercises_from_topsets(topsets),
        'Workouts': get_workouts(topsets)
    }

In [19]:
print(json.dumps(convert_to_view_model(topsets), indent=2))

{
  "PersonId": 1,
  "PersonName": "Gabe",
  "Exercises": [
    {
      "ExerciseId": 1,
      "ExerciseName": "Squat"
    },
    {
      "ExerciseId": 2,
      "ExerciseName": "Bench"
    },
    {
      "ExerciseId": 3,
      "ExerciseName": "Deadlift"
    },
    {
      "ExerciseId": 4,
      "ExerciseName": "Hotep"
    }
  ],
  "Workouts": [
    {
      "WorkoutId": 35,
      "StartDate": "2022-01-13",
      "TopSets": [
        {
          "TopSetId": 61,
          "ExerciseId": 2,
          "ExerciseName": "Bench",
          "Weight": 50,
          "Repetitions": 8
        }
      ]
    },
    {
      "WorkoutId": 36,
      "StartDate": "2022-01-18",
      "TopSets": [
        {
          "TopSetId": 62,
          "ExerciseId": 1,
          "ExerciseName": "Squat",
          "Weight": 55,
          "Repetitions": 7
        },
        {
          "TopSetId": 63,
          "ExerciseId": 4,
          "ExerciseName": "Hotep",
          "Weight": 25,
          "Repetitions": 4
        

# Dashboard page

In [20]:
all_topsets =query_db("""
    SELECT 
        P.PersonId, 
        P.Name AS PersonName, 
        W.WorkoutId, 
        W.StartDate, 
        T.TopSetId, 
        E.ExerciseId, 
        E.Name AS ExerciseName, 
        T.Repetitions, 
        T.Weight
    FROM Person P
         LEFT JOIN Workout W ON P.PersonId=W.PersonId
         LEFT JOIN TopSet T ON W.WorkoutId=T.WorkoutId
         LEFT JOIN Exercise E ON T.ExerciseId=E.ExerciseId""")

In [23]:
def get_people(topsets):
    # Get all unique workout_ids (No duplicates)
    people_ids = set([t['PersonId'] for t in topsets])

    # Group topsets into workouts
    people = []
    for person_id in people_ids:
        workouts_for_person = [t for t in topsets if t['PersonId'] == person_id]
        people.append({
            'PersonId': person_id,
            'PersonName': workouts_for_person[0]['PersonName'],
            'Workouts': get_workouts(workouts_for_person)
        })
    return people

In [24]:
print(json.dumps(get_people(all_topsets), indent=2))

[
  {
    "PersonId": 1,
    "PersonName": "Gabe",
    "Workouts": [
      {
        "WorkoutId": 35,
        "StartDate": "2022-01-13",
        "TopSets": [
          {
            "TopSetId": 61,
            "ExerciseId": 2,
            "ExerciseName": "Bench",
            "Weight": 50,
            "Repetitions": 8
          }
        ]
      },
      {
        "WorkoutId": 36,
        "StartDate": "2022-01-18",
        "TopSets": [
          {
            "TopSetId": 62,
            "ExerciseId": 1,
            "ExerciseName": "Squat",
            "Weight": 55,
            "Repetitions": 7
          },
          {
            "TopSetId": 63,
            "ExerciseId": 4,
            "ExerciseName": "Hotep",
            "Weight": 25,
            "Repetitions": 4
          }
        ]
      },
      {
        "WorkoutId": 37,
        "StartDate": "2022-02-08",
        "TopSets": [
          {
            "TopSetId": 64,
            "ExerciseId": 2,
            "ExerciseName": "Bench",


In [26]:
gabe_topsts = [t for t in topsets if t['PersonId'] == 1]

In [63]:
gabe_exercises = get_all_exercises_from_topsets(gabe_topsts)
gabe_rep_maxes = []
for e in gabe_exercises:
    exercise_topsets = [t for t in gabe_topsts if t['ExerciseId'] == e['ExerciseId']]
    set_reps = set([t['Repetitions'] for t in exercise_topsets])

    topsets_for_exercise = []
    for rep in set_reps:
        reps = [t for t in exercise_topsets if t['Repetitions'] == rep]
        max_weight = max([t['Weight'] for t in reps])
        max_topset_for_rep = [t for t in reps if t['Weight'] == max_weight]
        topsets_for_exercise.append({
            'StartDate': max_topset_for_rep[0]['StartDate'],
            'Repetitions': rep,
            'Weight': max_weight
        })
    
    topsets_for_exercise.sort(key=lambda x: datetime.strptime(x['StartDate'],"%Y-%m-%d"))
    
    gabe_rep_maxes.append({
            'ExerciseId': e['ExerciseId'],
            'ExerciseName': e['ExerciseName'],
            'RepMaxes': topsets_for_exercise,
        })

In [65]:
def get_rep_maxes_for_person(person_topsets):
    person_exercises = get_all_exercises_from_topsets(person_topsets)

    rep_maxes_in_exercises = []
    for e in person_exercises:
        exercise_topsets = [
            t for t in person_topsets if t['ExerciseId'] == e['ExerciseId']]
        set_reps = set([t['Repetitions'] for t in exercise_topsets])

        topsets_for_exercise = []
        for rep in set_reps:
            reps = [t for t in exercise_topsets if t['Repetitions'] == rep]
            max_weight = max([t['Weight'] for t in reps])
            max_topset_for_rep = [t for t in reps if t['Weight'] == max_weight]
            topsets_for_exercise.append({
                'StartDate': max_topset_for_rep[0]['StartDate'],
                'Repetitions': rep,
                'Weight': max_weight
            })

        topsets_for_exercise.sort(
            key=lambda x: datetime.strptime(x['StartDate'], "%Y-%m-%d"))

        rep_maxes_in_exercises.append({
            'ExerciseId': e['ExerciseId'],
            'ExerciseName': e['ExerciseName'],
            'RepMaxes': topsets_for_exercise,
        })
    return rep_maxes_in_exercises


In [66]:
get_rep_maxes_for_person(gabe_topsts)

[{'ExerciseId': 1,
  'ExerciseName': 'Squat',
  'RepMaxes': [{'StartDate': '2022-01-18', 'Repetitions': 7, 'Weight': 55},
   {'StartDate': '2022-03-15', 'Repetitions': 5, 'Weight': 75},
   {'StartDate': '2022-04-07', 'Repetitions': 6, 'Weight': 75},
   {'StartDate': '2022-04-26', 'Repetitions': 3, 'Weight': 80},
   {'StartDate': '2022-06-14', 'Repetitions': 2, 'Weight': 75}]},
 {'ExerciseId': 2,
  'ExerciseName': 'Bench',
  'RepMaxes': [{'StartDate': '2022-01-13', 'Repetitions': 8, 'Weight': 50},
   {'StartDate': '2022-02-08', 'Repetitions': 5, 'Weight': 55},
   {'StartDate': '2022-02-17', 'Repetitions': 2, 'Weight': 60},
   {'StartDate': '2022-03-10', 'Repetitions': 11, 'Weight': 50},
   {'StartDate': '2022-03-22', 'Repetitions': 4, 'Weight': 60},
   {'StartDate': '2022-04-21', 'Repetitions': 10, 'Weight': 45},
   {'StartDate': '2022-05-19', 'Repetitions': 1, 'Weight': 65}]},
 {'ExerciseId': 3,
  'ExerciseName': 'Deadlift',
  'RepMaxes': [{'StartDate': '2022-02-08', 'Repetitions': 6, 

In [67]:
def get_people_and_exercise_rep_maxes(topsets):
    # Get all unique workout_ids (No duplicates)
    people_ids = set([t['PersonId'] for t in topsets])

    # Group topsets into workouts
    people = []
    for person_id in people_ids:
        workouts_for_person = [t for t in topsets if t['PersonId'] == person_id]
        people.append({
            'PersonId': person_id,
            'PersonName': workouts_for_person[0]['PersonName'],
            'Exercises': get_rep_maxes_for_person(workouts_for_person)
        })
    return people

In [69]:
get_people_and_exercise_rep_maxes(all_topsets)

[{'PersonId': 1,
  'PersonName': 'Gabe',
  'Exercises': [{'ExerciseId': 1,
    'ExerciseName': 'Squat',
    'RepMaxes': [{'StartDate': '2022-01-18', 'Repetitions': 7, 'Weight': 55},
     {'StartDate': '2022-03-15', 'Repetitions': 5, 'Weight': 75},
     {'StartDate': '2022-04-07', 'Repetitions': 6, 'Weight': 75},
     {'StartDate': '2022-04-26', 'Repetitions': 3, 'Weight': 80},
     {'StartDate': '2022-06-14', 'Repetitions': 2, 'Weight': 75}]},
   {'ExerciseId': 2,
    'ExerciseName': 'Bench',
    'RepMaxes': [{'StartDate': '2022-01-13', 'Repetitions': 8, 'Weight': 50},
     {'StartDate': '2022-02-08', 'Repetitions': 5, 'Weight': 55},
     {'StartDate': '2022-02-17', 'Repetitions': 2, 'Weight': 60},
     {'StartDate': '2022-03-10', 'Repetitions': 11, 'Weight': 50},
     {'StartDate': '2022-03-22', 'Repetitions': 4, 'Weight': 60},
     {'StartDate': '2022-04-21', 'Repetitions': 10, 'Weight': 45},
     {'StartDate': '2022-05-19', 'Repetitions': 1, 'Weight': 65}]},
   {'ExerciseId': 3,
   